In [1]:
from utils import setup_env_path

setup_env_path()

%load_ext autoreload
%autoreload 2

Former working directory:  /Users/ambroisebertin/Desktop/prog/prog_abeilles/fil-rouge-pollinisateurs/notebooks
Current working directory:  /Users/ambroisebertin/Desktop/prog/prog_abeilles/fil-rouge-pollinisateurs


In [3]:
import pandas as pd
import numpy as np

In [2]:
API_keys = [
    "2b10Me1HF0rfjoGWCseolNa8e",
    "2b10Xs7brWPuBdRTSeWx9V7HJu",
    "2b10umm8L2jTYjlWrJKPfshJ0O",
    "2b10sEg5pSsrtVT372XqlrzcLe",
    "2b10LDWWlXJwuS6FleaqfK6Rke",
]

## Préparation des données

In [4]:
# Import spipoll.csv as a pandas dataframe
spipoll = pd.read_csv("././data/spipoll.csv",low_memory=False)

# Extract relevant columns from the spipoll dataframe
plantes = spipoll[['collection_id', 'plante_sc', 'plante_fr',
       'plante_precision', 'plante_inconnue', 'plante_caractere',
       'photo_fleur', 'photo_plante', 'photo_feuille']]

# Shrinking the data by grouping by collection_id
# Keep only the first row for each unique value of collection_id
plantes = plantes.drop_duplicates(subset='collection_id', keep='first')

plantes.shape

(75372, 9)

In [5]:
values_to_check = ["Je ne sais pas", "Plante inconnue"]

In [9]:
plantes_sc_unlabelled = plantes.loc[ #renommer en plantes_unlabelled_sc pour plus de clarté ??
    (
        (plantes["plante_sc"].isna() | plantes["plante_sc"].isin(values_to_check))
    )
    | ((plantes["plante_inconnue"] == 1.0))
]

# creation du dataframe contenant les valeurs de plantes mais pas de plantes_sc_unlabelled
plantes_sc_labelled = plantes[~plantes.index.isin(plantes_sc_unlabelled.index)]

# in plante add "1" to the column group for each row that is in plantes_sc_labelled
plantes.loc[plantes_sc_labelled.index, 'group'] = 1

plantes_sc_labelled.shape

(63465, 10)

In [10]:
# verification des dimensions des deux df par rapport à la dim de plantes
plantes_sc_labelled.shape[0] + plantes_sc_unlabelled.shape[0] == plantes.shape[0]

True

Compression de l'information étiquettée redondante :

In [11]:
# group plantes_sc_labelled by unique values of plante_sc
plantes_sc_labelled = plantes_sc_labelled.drop_duplicates(subset='plante_sc', keep='first')

# display the dimension of the dataframe
plantes_sc_labelled.shape

(2620, 10)

On a donc un df avec 2620 plantes dont le nom est spécifié dans plante_sc et 11.907 (~1.8% des données d'origine, spipoll.csv) dont le nom n'est pas spécifié dans plante_sc.

In [12]:
# ajoute trois colonnes après plante_sc dans le dataframe plantes_sc_labelled "Famille", "Genre" et "Espèce" entre la colonne plante_sc et plante_fr
plantes_sc_labelled.insert(2, "Famille", np.nan)
plantes_sc_labelled.insert(3, "Genre", np.nan)
plantes_sc_labelled.insert(4, "Espece", np.nan)

# ajoute trois colonnes après plante_sc dans le dataframe plantes_sc_unlabelled "Famille", "Genre" et "Espèce" entre la colonne plante_sc et plante_fr
plantes_sc_unlabelled.insert(2, "Famille", np.nan)
plantes_sc_unlabelled.insert(3, "Genre", np.nan)
plantes_sc_unlabelled.insert(4, "Espece", np.nan)

In [13]:
plantes_sc_labelled.head()

,collection_id,plante_sc,Famille,Genre,Espece,plante_fr,plante_precision,plante_inconnue,plante_caractere,photo_fleur,photo_plante,photo_feuille,group
0,1,Les Trèfles à fleurs blanches ou roses en boul...,NaN,NaN,NaN,NaN,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,1.0
2,2,Acanthus mollis,NaN,NaN,NaN,Acanthe molle,NaN,NaN,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,1.0
5,3,"Les Orchidées à fleurs blanches, jaunes ou ver...",NaN,NaN,NaN,NaN,orchis bouc - Himantoglossum hircinum,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,1.0
6,4,Les Vipérines à fleurs bleues-violacées (des e...,NaN,NaN,NaN,NaN,Theaceae,0.0,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,1.0
10,5,Les Pâquerettes (Bellis sp),NaN,NaN,NaN,NaN,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,1.0


## Appel à l'API